# try on kaggle

In [1]:
import os

import pandas as pd
import polars as pl

import kaggle_evaluation.aimo_2_inference_server
from transformers import AutoModelForCausalLM, AutoTokenizer

ModuleNotFoundError: No module named 'kaggle_evaluation'

In [ ]:
class TransformerSolver:
    def __init__(self, model_name, device="cuda"):
        self.model_name = model_name
        self.device = device
        self.max_new_tokens = 10000
        self.model = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype="auto",
            device_map="auto",
        )
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def solve(self, prompt, reasoning_type="CoT"):
        if reasoning_type == "CoT":
            messages = [
                {"role": "system", "content": "Please reason step by step, and put your final answer within \\boxed{}."},
                {"role": "user", "content": prompt}
            ]
        elif reasoning_type == "TIR":
            messages = [
                {"role": "system", "content": "Please integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}."},
                {"role": "user", "content": prompt}
            ]
        else:
            raise ValueError("Unsupported reasoning type")

        text = self.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = self.tokenizer([text], return_tensors="pt").to(self.device)

        generated_ids = self.model.generate(
            **model_inputs,
            max_new_tokens=self.max_new_tokens,
        )
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        return response

In [ ]:
model_name = "Qwen/Qwen2.5-Math-7B-Instruct"
device = "cuda" # the device to load the model onto
solver_Qwen_7B = TransformerSolver(model_name, device)
# response = solver_Qwen_7B.solve(prompt, reasoning_type="TIR")


In [ ]:
import re

def pred2int(pred):
    matches = re.findall(r'\\boxed{([^}]+)}', pred)
    tag = False
    for val in matches[-1::-1]:
        try:
            ans = int(val) % 1000
            tag = True
            break
        except:
            pass
    if tag:
        return ans
    else:
        return 0

In [4]:
# Replace this function with your inference code.
# The function should return a single integer between 0 and 999, inclusive.
# Each prediction (except the very first) must be returned within 30 minutes of the question being provided.
def predict(id_: pl.DataFrame, question: pl.DataFrame) -> pl.DataFrame | pd.DataFrame:
    """Make a prediction."""
    # Unpack values
    id_ = id_.item(0)
    question = question.item(0)
    # Make a prediction
    global solver_Qwen_7B
    response = solver_Qwen_7B.solve(question, reasoning_type="CoT")
    # trim the prediction
    ans = pred2int(response)
    return pl.DataFrame({'id': id_, 'answer': ans})

NameError: name 'pl' is not defined

In [ ]:
inference_server = kaggle_evaluation.aimo_2_inference_server.AIMO2InferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/ai-mathematical-olympiad-progress-prize-2/test.csv',
        )
    )